<a href="https://colab.research.google.com/github/vitoriaferreirap/DeepLearning/blob/main/CNN_Computer_Vision/3_TrainNetwork_VideoAnalysisUTFPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut for your standard (single animal) projects!

Some useful links:

- [DeepLabCut's GitHub: github.com/DeepLabCut/DeepLabCut](https://github.com/DeepLabCut/DeepLabCut)
- [DeepLabCut's Documentation: User Guide for Single Animal projects](https://deeplabcut.github.io/DeepLabCut/docs/standardDeepLabCut_UserGuide.html)


This notebook illustrates how to use the cloud to:
- create a training set
- train a network
- evaluate a network
- create simple quality check plots
- analyze novel videos!

### This notebook assumes you already have a project folder with labeled data!

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.


Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


https://github.com/DeepLabCut/DeepLabCut/blob/main/examples/COLAB/COLAB_YOURDATA_TrainNetwork_VideoAnalysis.ipynb

## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"

As the COLAB environments were updated to CUDA 12.X and Python 3.11, we need to install DeepLabCut and TensorFlow in a distinct way to get TensorFlow to connect to the GPU.

In [ ]:
# this will take a couple of minutes to install all the dependencies!
!pip install --pre deeplabcut

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 23.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━

**(Be sure to click "RESTART RUNTIME" if it is displayed above before moving on !)** You will see this button at the output of the cells above ^.

In [ ]:
import deeplabcut

Loading DLC 3.0.0rc13...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


## Link your Google Drive (with your labeled data, or the demo data):

### First, place your project folder into you google drive! "i.e. move the folder named "Project-YourName-TheDate" into google drive.

In [ ]:
# Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:
# (We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!

Typically, this will be: `/content/drive/My Drive/yourProjectFolderName`


In [ ]:
# PLEASE EDIT THIS:
project_folder_name = "deeplabcut/Cavalos-Poses-2026-02-24"
video_type = "mp4" #, mp4, MOV, or avi, whatever you uploaded!

# No need to edit this, we are going to assume you put videos you want to analyze
# in the "videos" folder, but if this is NOT true, edit below:
videofile_path = [f"/content/drive/My Drive/{project_folder_name}/videos/"]
print(videofile_path)

# The prediction files and labeled videos will be saved in this `labeled-videos` folder
# in your project folder; if you want them elsewhere, you can edit this;
# if you want the output files in the same folder as the videos, set this to an empty string.
destfolder = f"/content/drive/My Drive/{project_folder_name}/labeled-videos"

#No need to edit this, as you set it when you passed the ProjectFolderName (above):
path_config_file = f"/content/drive/My Drive/{project_folder_name}/config.yaml"
print(path_config_file)

# This creates a path variable that links to your Google Drive project

['/content/drive/My Drive/deeplabcut/Cavalos-Poses-2026-02-24/videos/']
/content/drive/My Drive/deeplabcut/Cavalos-Poses-2026-02-24/config.yaml


## Create a training dataset:

### You must do this step inside of Colab

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models-pytorch** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pytorch_config.yaml**.

Now it is the time to start training the network!

In [ ]:
# There are many more functions you can set here, including which network to use!
# Check the docstring for `create_training_dataset` for all options you can use!

deeplabcut.create_training_dataset(
  path_config_file, net_type="resnet_50", engine=deeplabcut.Engine.PYTORCH
)

[(0.95, 1, (array([2, 3, 1]), array([0])))]

## Start training:
This function trains the network for a specific shuffle of the training dataset.

In [ ]:
# Let's also change the display and save_epochs just in case Colab takes away
# the GPU... If that happens, you can reload from a saved point using the
# `snapshot_path` argument to `deeplabcut.train_network`:
#   deeplabcut.train_network(..., snapshot_path="/content/.../snapshot-050.pt")

# Typically, you want to train to ~200 epochs. We set the batch size to 8 to
# utilize the GPU's capabilities.

# More info and there are more things you can set:
#   https://deeplabcut.github.io/DeepLabCut/docs/standardDeepLabCut_UserGuide.html#g-train-the-network

deeplabcut.train_network(
    path_config_file,
    shuffle=1,
    save_epochs=5,
    epochs=200,
    batch_size=8,
)

# This will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end.

Training with configuration:
data:
  bbox_margin: 20
  colormode: RGB
  inference:
    normalize_images: True
  train:
    affine:
      p: 0.5
      rotation: 30
      scaling: [0.5, 1.25]
      translation: 0
    crop_sampling:
      width: 448
      height: 448
      max_shift: 0.1
      method: hybrid
    gaussian_noise: 12.75
    motion_blur: True
    normalize_images: True
device: auto
inference:
  multithreading:
    enabled: True
    queue_length: 4
    timeout: 30.0
  compile:
    enabled: False
    backend: inductor
  autocast:
    enabled: False
metadata:
  project_path: /content/drive/My Drive/deeplabcut/Cavalos-Poses-2026-02-24
  pose_config_path: /content/drive/My Drive/deeplabcut/Cavalos-Poses-2026-02-24/dlc-models-pytorch/iteration-0/CavalosFeb24-trainset95shuffle1/train/pytorch_config.yaml
  bodyparts: ['Ext_Empadua', 'Ext_Codilho', 'Ext_Joelho', 'Ext_CanelaDianteira', 'Ext_CascoDianteiro', 'Ext_Coxa', 'Ext_Perna', 'Ext_Jarrete', 'Ext_CanelaTraseira', 'Ext_CascoTraseir

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

[timm/resnet50_gn.a1h_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
Data Transforms:
  Training:   Compose([
  Affine(always_apply=False, p=0.5, interpolation=1, mask_interpolation=0, cval=0, mode=0, scale={'x': (0.5, 1.25), 'y': (0.5, 1.25)}, translate_percent=None, translate_px={'x': (0, 0), 'y': (0, 0)}, rotate=(-30, 30), fit_output=False, shear={'x': (0.0, 0.0), 'y': (0.0, 0.0)}, cval_mask=0, keep_ratio=True, rotate_method='largest_box'),
  PadIfNeeded(always_apply=True, p=1.0, min_height=448, min_width=448, pad_height_divisor=None, pad_width_divisor=None, position=PositionType.CENTER, border_mode=0, value=None, mask_value=None),
  KeypointAwareCrop(always_apply=True, p=1.0, width=448, height=448, max_shift=0.1, crop_sampling='hybrid'),
  MotionBlur(always_apply=False, p=0.5, blur_limit=(3, 7), allow_shifted=True),
  GaussNoise(always_apply=False, p=0.5, var_limit=(0, 162.5625), per_channel=True, mean=0),
  Nor

Note, that **when you hit "STOP" you will get a `KeyboardInterrupt` "error"! No worries! :)**

## Start evaluating:
This function evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results-pytorch**

In [ ]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as
# good as the labeler, so be sure your labels are good!


Evaluation scorer: DLC_Resnet50_CavalosFeb24shuffle1_snapshot_best-50


100%|██████████| 1/1 [00:00<00:00, 31.49it/s]


Evaluation results file: DLC_Resnet50_CavalosFeb24shuffle1_snapshot_best-50-results.csv
Evaluation results for DLC_Resnet50_CavalosFeb24shuffle1_snapshot_best-50-results.csv (pcutoff: 0.6):
train rmse             6.85
train rmse_pcutoff     6.19
train mAP             82.28
train mAR             83.33
test rmse              6.24
test rmse_pcutoff      5.96
test mAP              80.00
test mAR              80.00
Name: (0.95, 1, 50, -1, 0.6), dtype: float64


## There is an optional refinement step you can do outside of Colab:
- if your pixel errors are not low enough, please check out the protocol guide on how to refine your network!
- You will need to adjust the labels **outside of Colab!** We recommend coming back to train and analyze videos...
- Please see the repo and protocol instructions on how to refine your data!

## Start Analyzing videos:
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides.

In [ ]:
deeplabcut.analyze_videos(
    path_config_file,
    videofile_path,
    videotype=video_type,
    destfolder=destfolder,
)

Creating the output folder /content/drive/My Drive/deeplabcut/Cavalos-Poses-2026-02-24/labeled-videos
Analyzing videos with /content/drive/My Drive/deeplabcut/Cavalos-Poses-2026-02-24/dlc-models-pytorch/iteration-0/CavalosFeb24-trainset95shuffle1/train/snapshot-best-050.pt
Using scorer: DLC_Resnet50_CavalosFeb24shuffle1_snapshot_best-50
Starting to analyze /content/drive/My Drive/deeplabcut/Cavalos-Poses-2026-02-24/videos/video1.mp4
Video metadata: 
  Overall # of frames:    98
  Duration of video [s]:  3.92
  fps:                    25.0
  resolution:             w=848, h=478

Running pose prediction with batch size 8


100%|██████████| 98/98 [00:04<00:00, 20.93it/s]


Saving results in /content/drive/My Drive/deeplabcut/Cavalos-Poses-2026-02-24/labeled-videos/video1DLC_Resnet50_CavalosFeb24shuffle1_snapshot_best-50.h5 and /content/drive/My Drive/deeplabcut/Cavalos-Poses-2026-02-24/labeled-videos/video1DLC_Resnet50_CavalosFeb24shuffle1_snapshot_best-50_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.



'DLC_Resnet50_CavalosFeb24shuffle1_snapshot_best-50'

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
deeplabcut.plot_trajectories(
    path_config_file,
    videofile_path,
    videotype=video_type,
    destfolder=destfolder,
)

Analyzing all the videos in the directory...
Loading  /content/drive/My Drive/deeplabcut/Cavalos-Poses-2026-02-24/videos/video1.mp4 and data.
Plots created! Please check the directory "plot-poses" within the video directory


Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4.

## Create labeled video:
This function is for visualization purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.

In [ ]:
deeplabcut.create_labeled_video(
    path_config_file,
    videofile_path,
    videotype=video_type,
    destfolder=destfolder,
)

Analyzing all the videos in the directory...
Starting to process video: /content/drive/My Drive/deeplabcut/Cavalos-Poses-2026-02-24/videos/video1.mp4
Loading /content/drive/My Drive/deeplabcut/Cavalos-Poses-2026-02-24/videos/video1.mp4 and data.
Duration of video [s]: 3.92, recorded with 25.0 fps!
Overall # of frames: 98 with cropped frame dimensions: 848 478
Generating frames and creating video.


100%|██████████| 98/98 [00:00<00:00, 118.14it/s]


[True]